# store

> Store text features to avoid recomputing them.

In [ ]:
#| default_exp store

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pickle
import os
import hashlib
import sqlite3
import string

In [ ]:
#| export
class TextFeatureStore:
	""" A class to store features extracted for a text classification pipeline and cache them to disk to avoid recomputing them. """
	def __init__(self, 
			  	 path:str  # where to save sqllite db to persist the feature store between runs
				 ):
		self.texts = {}
		self.path = path

		if self.path is not None:
			if os.path.dirname(self.path) != '':
				# self.path shouuld be a file - make dirs if it doesn't exist
				if not os.path.exists(os.path.dirname(self.path)):
					os.makedirs(os.path.dirname(self.path))
		else:
			raise ValueError("The feature store requires a path")

		with sqlite3.connect(self.path) as conn:
			#conn.execute("PRAGMA journal_mode=WAL")
			conn.execute('''CREATE TABLE IF NOT EXISTS texts
				(hash TEXT PRIMARY KEY, tokens BLOB NOT NULL, pos BLOB NOT NULL, textstats BLOB NOT NULL) WITHOUT ROWID;''')
			conn.execute('''CREATE TABLE IF NOT EXISTS embeddings
				(hash TEXT PRIMARY KEY, embeddings BLOB NOT NULL) WITHOUT ROWID;''')		
			conn.execute('''CREATE TABLE IF NOT EXISTS lexicons
				(hash TEXT PRIMARY KEY, lexicons BLOB NOT NULL) WITHOUT ROWID;''')	
			conn.commit()

	def empty(self):
		"""Clear the contents of the feature store."""
		with sqlite3.connect(self.path) as conn:
			conn.execute('''DELETE FROM texts;''')
			conn.execute('''DELETE FROM embeddings;''')
			conn.execute('''DELETE FROM lexicons;''')
			conn.commit()

	# def save(self):
	# 	"""Save the feature store to disk."""
	# 	if self.path is not None:
	# 		with open(self.path, 'wb') as f:
	# 			pickle.dump(self.texts, f)

	def update(self, 
			   text:str, # the text to update
			   tokens:list, # the tokens to update
			   pos:list, # the part of speech tags to update
			   textstats:list, # the text statistics to update
			   ):
		"""Update the feature store tokens, parts of speech tags and text statistics for a text."""
		hash = hashlib.md5(text.encode()).hexdigest()
		with sqlite3.connect(self.path) as conn:
			conn.execute('''
				INSERT OR REPLACE INTO texts (hash, tokens, pos, textstats)
				VALUES (?, ?, ?, ?)
			''', (hash, pickle.dumps(tokens), pickle.dumps(pos), pickle.dumps(textstats)))
			conn.commit()

	def update_embeddings(self, 
					   	  texts:str, # the texts to update 
						  embeddings:list # the embeddings to update
						  ):
		"""Update the feature store with embeddings for a list of texts."""

		with sqlite3.connect(self.path) as conn:
			for i, text in enumerate(texts):
				hash = hashlib.md5(text.encode()).hexdigest()
				conn.execute('''
					INSERT OR REPLACE INTO embeddings (hash, embeddings) 
					VALUES (?, ?)
				''', (hash, pickle.dumps(embeddings[i])))
				conn.commit()

	def update_lexicons(self, 
					   	  texts:str, # the texts to update 
						  lexicons:list # the lexicon scores to update
						  ):
		"""Update the feature store with lexicon features for a list of texts."""

		with sqlite3.connect(self.path) as conn:
			for i, text in enumerate(texts):
				hash = hashlib.md5(text.encode()).hexdigest()
				conn.execute('''
					INSERT OR REPLACE INTO lexicons (hash, lexicons) 
					VALUES (?, ?)
				''', (hash, pickle.dumps(lexicons[i])))
				conn.commit()

	def get(self, 
		 	text:str, # the text to get features for
			type:str = None # the type of features to get - 'tokens', 'pos', 'textstats', 'embeddings', 'lexicons'
			) -> dict|list: # the features for the text
		""" Get features for a text."""

		if type is not None and type not in ['tokens', 'pos', 'textstats', 'embeddings', 'lexicons']:
			raise ValueError(f"Type {type} not in ['tokens', 'pos', 'textstats', 'embeddings', 'lexicons']")

		with sqlite3.connect(self.path) as conn:
			hash = hashlib.md5(text.encode()).hexdigest()
			if type is None:
				cursor = conn.execute('SELECT * FROM texts WHERE hash=?', (hash,))
				row = cursor.fetchone()
				result = {'tokens': pickle.loads(row[1]), 'pos': pickle.loads(row[2]), 'textstats': pickle.loads(row[3])}
				if row is None:
					return None
				else:
					cursor = conn.execute('SELECT * FROM embeddings WHERE hash=?', (hash,))
					row = cursor.fetchone()
					if row is None:
						result['embeddings'] = None
					else:
						result['embeddings'] = pickle.loads(row[1])

					cursor = conn.execute('SELECT * FROM lexicons WHERE hash=?', (hash,))
					row = cursor.fetchone()
					if row is None:
						result['lexicons'] = None
					else:
						result['lexicons'] = pickle.loads(row[1])
					return result

			elif type in ['tokens', 'pos', 'textstats']:
				cursor = conn.execute(f'SELECT {type} FROM texts WHERE hash=?', (hash,))
				row = cursor.fetchone()
				if row is None:
					return None
				else:
					return pickle.loads(row[0])
			elif type == 'embeddings':
				cursor = conn.execute('SELECT embeddings FROM embeddings WHERE hash=?', (hash,))
				row = cursor.fetchone()
				if row is None:
					return None
				else:
					return pickle.loads(row[0])
			elif type == 'lexicons':
				cursor = conn.execute('SELECT lexicons FROM lexicons WHERE hash=?', (hash,))
				row = cursor.fetchone()
				if row is None:
					return None
				else:
					return pickle.loads(row[0])

	def get_features_from_texts_by_type(self,
							 texts:list, # the texts to get features for
							 type:str # the type of features to get - 'tokens', 'pos', 'textstats', 'embeddings', 'lexicons'
							 ) -> list: # the features for the texts
		""" Get features for a list of texts."""
		hashes = [hashlib.md5(text.encode()).hexdigest() for text in texts]

		if type not in ['tokens', 'pos', 'textstats', 'embeddings', 'lexicons']:
			raise ValueError(f"Type {type} not in ['tokens', 'pos', 'textstats', 'embeddings', 'lexicons']")

		with sqlite3.connect(self.path) as conn:
			if type == 'embeddings':
				cursor = conn.execute(f'SELECT hash, embeddings FROM embeddings WHERE hash IN ({", ".join(["?"] * len(hashes))})', hashes)
			elif type == 'lexicons':
				cursor = conn.execute(f'SELECT hash, lexicons FROM lexicons WHERE hash IN ({", ".join(["?"] * len(hashes))})', hashes)
			else:
				cursor = conn.execute(f'SELECT hash, {type} FROM texts WHERE hash IN ({", ".join(["?"] * len(hashes))})', hashes)
			features = [None] * len(hashes)

			positions = {}
			for i, hash in enumerate(hashes):
				if hash not in positions:
					positions[hash] = []
				positions[hash].append(i)

			for row in cursor:
				if row[0] in hashes:
					for position in positions[row[0]]:
						features[position] = pickle.loads(row[1])
				else:
					raise ValueError(f"Hash {row[0]} not in {hashes}")
			
		return features

	def get_tokens_from_texts(self, 
						      texts:list, # the texts to get tokens for
							  lowercase:bool = False, # whether to lowercase the tokens
							  min_token_length:int = 0, # the minimum token length to include
							  remove_punctuation:bool = False, # whether to remove punctuation
							  remove_numbers:bool = False, # whether to remove numbers
							  ) -> list: # the tokens for the texts
		""" Get tokens for a list of texts. """
		tokens = self.get_features_from_texts_by_type(texts, 'tokens')
		if lowercase == True:
			tokens = [[token.lower() for token in text] for text in tokens]
		if min_token_length > 0:
			tokens = [[token for token in text if len(token) >= min_token_length] for text in tokens]
		if remove_punctuation == True:
			tokens = [[token for token in text if token.strip(string.punctuation)] for text in tokens]
		if remove_numbers == True:
			tokens = [[token for token in text if not token.isdigit()] for text in tokens]
		return tokens
	
	def get_textstats_from_texts(self, 
							  	 texts:list, # the texts to get text statistics for
								 columns_out = ['monosyll_count', 'polysyll_count', 'token_count', 'sentence_count', 'unique_tokens_count', 'average_sentence_length'], # the text statistics to get
								 columns_in = ['monosyll_count', 'polysyll_count', 'token_count', 'sentence_count', 'unique_tokens_count', 'average_sentence_length'] # the column name order - leaving for future use, but probably not needed
								 ) -> list: # the text statistics for the texts
		""" Get text statistics for a list of texts. """
		try:
			columns = [columns_in.index(col) for col in columns_out]
		except ValueError:
			raise ValueError(f"Columns {columns_out} not in {columns_in}")

		textstats = self.get_features_from_texts_by_type(texts, 'textstats')
		textstats = [[textstat[i] for i in columns] for textstat in textstats]
		return textstats
	
	def get_pos_from_texts(self, 
						   texts:list, # the texts to get part of speech tags for
						   ) -> list: # the part of speech tags for the texts
		""" Get parts of speech for a list of texts. """
		pos = self.get_features_from_texts_by_type(texts, 'pos')
		return pos

	def get_embeddings_from_texts(self, 
							      texts:str # the texts to get embeddings for
								  ) -> list: # the embeddings for the texts
		""" Get embeddings for multiple texts. """

		embeddings = self.get_features_from_texts_by_type(texts, 'embeddings')
		return embeddings
	
	def get_lexicons_from_texts(self,
							 texts:str # the texts to get lexicons for
							 ) -> list:
		""" Get lexicons for multiple texts. """
		lexicons = self.get_features_from_texts_by_type(texts, 'lexicons')
		return lexicons
	
	def dump(self):
		""" show the contents of the feature store (intended for debugging/development) """
		with sqlite3.connect(self.path) as conn:
			cursor = conn.execute('SELECT * FROM texts')
			for row in cursor:
				print(row[0], pickle.loads(row[1]), pickle.loads(row[2]), pickle.loads(row[3]))
			cursor = conn.execute('SELECT * FROM embeddings')
			for row in cursor:
				print(row[0], pickle.loads(row[1]))
			cursor = conn.execute('SELECT * FROM lexicons')
			for row in cursor:
				print(row[0], pickle.loads(row[1]))


TODO: add an example.

In [ ]:
#| hide
text_feature_store = TextFeatureStore('test_store.sqlite')

assert os.path.exists('test_store.sqlite')

text_feature_store.empty()
text_feature_store.update('Hello, world!', ['Hello', 'world'], ['INTJ', 'NOUN'], [1, 1])

assert text_feature_store.get('Hello, world!') == {'tokens': ['Hello', 'world'], 'pos': ['INTJ', 'NOUN'], 'textstats': [1, 1], 'embeddings': None, 'lexicons': None}
assert text_feature_store.get('Hello, world!', 'tokens') == ['Hello', 'world']
assert text_feature_store.get('Hello, world!', 'pos') == ['INTJ', 'NOUN']
assert text_feature_store.get('Hello, world!', 'textstats') == [1, 1]

text_feature_store.update_embeddings(['Hello, world!'], [[1, 2, 3]])
assert text_feature_store.get('Hello, world!', 'embeddings') == [1, 2, 3]

text_feature_store.update_lexicons(['Hello, world!'], [[9, 11, 13]])
assert text_feature_store.get('Hello, world!', 'lexicons') == [9, 11, 13]

text_feature_store.update('Hello, world 123!', ['Hello', 'world', '123'], ['INTJ', 'NOUN'], [1, 1])
text_feature_store.update('Hello, world 456!', ['Hello', 'world', '456'], ['INTJ', 'NOUN'], [1, 1])
assert text_feature_store.get_tokens_from_texts(['Hello, world!', 'Hello, world 123!', 'Hello, world 456!']) == [['Hello', 'world'], ['Hello', 'world', '123'], ['Hello', 'world', '456']]

text_feature_store.empty()
assert text_feature_store.get('Hello, world!', 'tokens') is None
assert text_feature_store.get('Hello, world!', 'pos') is None
assert text_feature_store.get('Hello, world!', 'textstats') is None
assert text_feature_store.get_embeddings_from_texts(['Hello, world!']) == [None]
assert text_feature_store.get_lexicons_from_texts(['Hello, world!']) == [None]

text_feature_store.update('Hello, world!', ['Hello', 'world'], ['INTJ', 'NOUN'], [1, 1])
text_feature_store.update_embeddings(['Hello, world!'], [[1, 2, 3]])
assert text_feature_store.get_embeddings_from_texts(['Hello, world!']) == [[1, 2, 3]]
assert text_feature_store.get_embeddings_from_texts(['Hello, world!', 'Hi, world!']) == [[1, 2, 3], None]

text_feature_store.update_lexicons(['Hello, world!'], [[9, 11, 13]])
assert text_feature_store.get_lexicons_from_texts(['Hello, world!']) == [[9, 11, 13]]
assert text_feature_store.get_lexicons_from_texts(['Hello, world!', 'Hi, world!']) == [[9, 11, 13], None]

text_feature_store.empty()
assert text_feature_store.get('Hello, world!', 'tokens') is None

text_feature_store.update('A Hello, world!', ['A', 'Hello', ',', 'world', '!'], ['INTJ', 'PUNCT', 'NOUN', 'PUNCT'], [1, 1, 2, 1, 2, 1])
assert text_feature_store.get_tokens_from_texts(['A Hello, world!']) == [['A', 'Hello', ',', 'world', '!']]
assert text_feature_store.get_tokens_from_texts(['A Hello, world!'], lowercase=True) == [['a', 'hello', ',', 'world', '!']]
assert text_feature_store.get_tokens_from_texts(['A Hello, world!'], min_token_length=2) == [['Hello', 'world']]
assert text_feature_store.get_tokens_from_texts(['A Hello, world!'], remove_punctuation=True) == [['A', 'Hello', 'world']]

text_feature_store.empty()
text_feature_store.update('', [], [], [0, 0, 0, 0, 0, 0])
assert text_feature_store.get('') == {'tokens': [], 'pos': [], 'textstats': [0, 0, 0, 0, 0, 0], 'embeddings': None, 'lexicons': None}

# text_feature_store = TextFeatureStore()
# text_feature_store.update('Hello, world!', ['Hello', 'world'], ['NNP', 'NN'], [1, 1])
# assert text_feature_store.get('Hello, world!') == {'tokens': ['Hello', 'world'], 'pos': ['NNP', 'NN'], 'textstats': [1, 1]}
# text_feature_store.save()
# text_feature_store = text_feature_store.load()
# assert text_feature_store.get('Hello, world!') == {'tokens': ['Hello', 'world'], 'pos': ['NNP', 'NN'], 'textstats': [1, 1]}

text_feature_store.empty()
text_feature_store.update('Hello, world!', ['Hello', 'world'], ['INTJ', 'NOUN'], [1, 1, 2, 1, 2, 1])
assert text_feature_store.get_textstats_from_texts(['Hello, world!']) == [[1, 1, 2, 1, 2, 1]]
assert text_feature_store.get_textstats_from_texts(['Hello, world!'], columns_out=['monosyll_count', 'polysyll_count']) == [[1, 1]]

# this should raise a value error
try:
    textstats = text_feature_store.get_textstats_from_texts(['Hello, world!'], columns_out=['dr_dre'])
except ValueError as e:
    assert True
else:
    assert False, "Expected ValueError not raised"

del text_feature_store
os.remove('test_store.sqlite')


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()